In [1]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np



In [2]:
pip install keras_preprocessing

Note: you may need to restart the kernel to use updated packages.


In [3]:
TRAIN_DIR = 'finaldataset/train'
TEST_DIR = 'finaldataset/test'


In [4]:
import os

def createdataframe(directory):
    image_paths = []
    labels = []
    
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        
        # Skip if not a directory
        if not os.path.isdir(label_dir):
            continue
        
        for imagename in os.listdir(label_dir):
            # Skip hidden files or non-image files
            if imagename.startswith('.') or not imagename.endswith(('.jpg', '.jpeg', '.png')):
                continue
            
            image_paths.append(os.path.join(label_dir, imagename))
            labels.append(label)
    
    return image_paths, labels


In [5]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

In [6]:
print(train)

                                    image  label
0       finaldataset/train/happy/3578.jpg  happy
1      finaldataset/train/happy/16988.jpg  happy
2       finaldataset/train/happy/2666.jpg  happy
3       finaldataset/train/happy/5109.jpg  happy
4      finaldataset/train/happy/11981.jpg  happy
...                                   ...    ...
22142    finaldataset/train/angry/184.jpg  angry
22143  finaldataset/train/angry/14638.jpg  angry
22144  finaldataset/train/angry/15526.jpg  angry
22145  finaldataset/train/angry/24923.jpg  angry
22146  finaldataset/train/angry/12249.jpg  angry

[22147 rows x 2 columns]


In [7]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

In [8]:
print(test)

                                  image  label
0     finaldataset/test/happy/23933.jpg  happy
1     finaldataset/test/happy/24906.jpg  happy
2     finaldataset/test/happy/18033.jpg  happy
3     finaldataset/test/happy/15271.jpg  happy
4     finaldataset/test/happy/13833.jpg  happy
...                                 ...    ...
3653  finaldataset/test/angry/19485.jpg  angry
3654   finaldataset/test/angry/9931.jpg  angry
3655  finaldataset/test/angry/18994.jpg  angry
3656   finaldataset/test/angry/1438.jpg  angry
3657  finaldataset/test/angry/10338.jpg  angry

[3658 rows x 2 columns]


In [9]:
from tqdm.notebook import tqdm

In [10]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale =  True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features
    


In [11]:
pip install Pillow

Note: you may need to restart the kernel to use updated packages.


In [12]:
train_features = extract_features(train['image']) 

  0%|          | 0/22147 [00:00<?, ?it/s]

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [13]:
test_features = extract_features(test['image'])

  0%|          | 0/3658 [00:00<?, ?it/s]

In [14]:
x_train = train_features/255.0
x_test = test_features/255.0

In [15]:
pip install -U scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 29.1 MB/s eta 0:00:000:00:0136m0:00:01:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.4.1.post1
    Uninstalling scikit-learn-1.4.1.post1:
      Successfully uninstalled scikit-learn-1.4.1.post1
Note: you may need to restart the kernel to use updated packages.


In [16]:
from sklearn.preprocessing import LabelEncoder

In [17]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [18]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [19]:
y_train = to_categorical(y_train,num_classes = 5)
y_test = to_categorical(y_test,num_classes = 5)

In [20]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(5, activation='softmax'))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [21]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [22]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 20, validation_data = (x_test,y_test)) 

Epoch 1/20
174/174 ━━━━━━━━━━━━━━━━━━━━ 177s 1s/step - accuracy: 0.3051 - loss: 1.5816 - val_accuracy: 0.2507 - val_loss: 1.6189
Epoch 2/20
174/174 ━━━━━━━━━━━━━━━━━━━━ 239s 1s/step - accuracy: 0.3130 - loss: 1.5728 - val_accuracy: 0.2507 - val_loss: 1.6300
Epoch 3/20
174/174 ━━━━━━━━━━━━━━━━━━━━ 844s 5s/step - accuracy: 0.3136 - loss: 1.5585 - val_accuracy: 0.3403 - val_loss: 1.5116
Epoch 4/20
174/174 ━━━━━━━━━━━━━━━━━━━━ 271s 2s/step - accuracy: 0.3716 - loss: 1.4672 - val_accuracy: 0.4087 - val_loss: 1.3762
Epoch 5/20
174/174 ━━━━━━━━━━━━━━━━━━━━ 166s 955ms/step - accuracy: 0.4209 - loss: 1.3799 - val_accuracy: 0.4483 - val_loss: 1.3442
Epoch 6/20
174/174 ━━━━━━━━━━━━━━━━━━━━ 369s 2s/step - accuracy: 0.4614 - loss: 1.2992 - val_accuracy: 0.4913 - val_loss: 1.2158
Epoch 7/20
174/174 ━━━━━━━━━━━━━━━━━━━━ 163s 937ms/step - accuracy: 0.4790 - loss: 1.2543 - val_accuracy: 0.4984 - val_loss: 1.2217
Epoch 8/20
174/174 ━━━━━━━━━━━━━━━━━━━━ 253s 1s/step - accuracy: 0.4975 - loss: 1.2153 - va

In [23]:
from keras.models import model_from_json


In [24]:
model_json = model.to_json()
with open("facialemotionrecognition.json",'w') as json_file:
    json_file.write(model_json)
model.save("facialemotionrecognition.h5")